In [6]:
import numpy as np
y = [[2,3,4,5,6,7,8,9], [2,3,4,5,6,7,8,9], [2,3,4,5,6,7,8,9]]
x = np.array(y)
print(x.shape)
print(np.sum(x))

(3, 8)
132


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

from google.colab import drive
drive.mount('/content/drive')
# # !ls 'content/drive/MyDrive/'
# import csv

with open('/content/drive/MyDrive/train.csv', mode = 'r') as file:
  # trainfile = csv.reader(file)
  trainfile = pd.read_csv(file)


# Missing value columns are identified.
missing_val_col_list = []
col = trainfile.shape[1]-1


# /// 1. Find and modify the missing values in the data file. \\\

for i in range(1,col):
  if(trainfile.iloc[:, i].isnull().sum()):
    missing_val_col_list.append(i)
# print(missing_val_col_list)


# Segregating classification type and numerical type missing value column.
missing_val_ctype_col_list = []
missing_val_ntype_col_list = []
for j in missing_val_col_list:
  if trainfile.iloc[:, j].dtype == 'float':
    missing_val_ntype_col_list.append(j)
  else:
    missing_val_ctype_col_list.append(j)
print("///////1. trainfile\\\\\\")
print("Classification type column->", missing_val_ctype_col_list)
print("Numerical type column->", missing_val_ntype_col_list)


# Listing the missing data percentage of the column and removing  if more than 45% is missing.
missing_percent = trainfile.iloc[:,1:col].isnull().mean()*100
for i in missing_percent:
  if i > 0:
    print(i)
trainfile_cleaned = trainfile.drop(columns = missing_percent[missing_percent>45].index)

missing_val_col = []
mod_col = trainfile_cleaned.shape[1]
for i in range(1,mod_col):
  if(trainfile_cleaned.iloc[:, i].isnull().sum()):
    missing_val_col.append(i)
# print(missing_val_col)

missing_val_ctype_col = []
missing_val_ntype_col = []
for j in missing_val_col:
  if trainfile_cleaned.iloc[:, j].dtype == 'float':
    missing_val_ntype_col.append(j)
  else:
    missing_val_ctype_col.append(j)
print("///////2. trainfile_cleaned\\\\\\")
print("Modified Classification type column->", missing_val_ctype_col)
print("Modified Numerical type column->", missing_val_ntype_col)

missing_percentage = trainfile_cleaned.iloc[:,1:mod_col].isnull().mean()*100
for i in missing_percentage:
  if i > 0:
    print(i)


# /// 2. Filling up missing columns with median(for numerical data) and mode(for classification data).\\\

print("///////3. trainfile_cleaned\\\\\\")
print("")
print("Classification data sets.")
print("")
for k in missing_val_ctype_col:
  trainfile_cleaned.iloc[:, k] = trainfile_cleaned.iloc[:, k].fillna(trainfile_cleaned.iloc[:, k].mode()[0])
  print(trainfile_cleaned.iloc[:, k].describe(include = ['object']))

print("")
print("Numerical data sets.")
print("")
for k in missing_val_ntype_col:
  trainfile_cleaned.iloc[:, k] = trainfile_cleaned.iloc[:, k].fillna(trainfile_cleaned.iloc[:, k].median())
  print(trainfile_cleaned.iloc[:, k].describe())

for i in range(len(missing_val_ntype_col)):
  # sns.histplot(trainfile_cleaned.iloc[:, missing_val_ntype_col[i]], kde = True, bins = 120)
  # sns.boxplot(x = trainfile_cleaned.iloc[:, missing_val_ntype_col[i]])
  y_value = trainfile_cleaned.iloc[:, missing_val_ntype_col[i]]
  x_value = range(len(y_value))
  plt.scatter(x_value, y_value, color = 'blue', alpha = 0.5)
  plt.show()


# \\\DEBUG CODE/// ->To check that all missing data's are filled.
# print("//////////////////////////////////////////////////////////////////////////////")
print("///////4. trainfile_cleaned\\\\\\")
print("")
print("Classification data sets.")
print("")
for k in missing_val_ctype_col:
  print(trainfile_cleaned.iloc[:, k].describe(include = ['object']))
print("")
print("Numerical data sets.")
print("")
for k in missing_val_ntype_col:
  print(trainfile_cleaned.iloc[:, k].describe())


classification_col = trainfile_cleaned.select_dtypes(include=['object', 'category']).columns
classification_indices = [trainfile_cleaned.columns.get_loc(col) for col in classification_col]

numerical_col = trainfile_cleaned.select_dtypes(include=['number']).columns
numerical_indices = [trainfile_cleaned.columns.get_loc(col) for col in numerical_col]

print(f"{classification_indices} total: {len(classification_indices)}")

for i in range(len(classification_col)):
  x = trainfile_cleaned[numerical_col[len(numerical_col) - 1]]
  y = trainfile_cleaned[classification_col[i]]
  plt.scatter(y, x, color = 'blue', alpha = 0.5)
  plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Read the data
with open('/content/drive/MyDrive/train.csv', mode='r') as file:
    trainfile = pd.read_csv(file)

# Identifying columns with missing values
missing_val_col_list = []
col = trainfile.shape[1] - 1

for i in range(1, col):
    if trainfile.iloc[:, i].isnull().sum():
        missing_val_col_list.append(i)

# Separating numerical and categorical columns
missing_val_ctype_col_list = []
missing_val_ntype_col_list = []
for j in missing_val_col_list:
    if pd.api.types.is_numeric_dtype(trainfile.iloc[:, j]):
        missing_val_ntype_col_list.append(j)
    else:
        missing_val_ctype_col_list.append(j)

# Calculate the percentage of missing values and drop columns with >45% missing values
missing_percent = trainfile.iloc[:, 1:col].isnull().mean() * 100
trainfile_cleaned = trainfile.drop(columns=missing_percent[missing_percent > 45].index)

# Identify the columns with remaining missing values in the cleaned dataset
missing_val_col = []
mod_col = trainfile_cleaned.shape[1]

for i in range(1, mod_col):
    if trainfile_cleaned.iloc[:, i].isnull().sum():
        missing_val_col.append(i)

# Separating numerical and categorical columns again
missing_val_ctype_col = []
missing_val_ntype_col = []
for j in missing_val_col:
    if pd.api.types.is_numeric_dtype(trainfile_cleaned.iloc[:, j]):
        missing_val_ntype_col.append(j)
    else:
        missing_val_ctype_col.append(j)

# Fill missing values for categorical and numerical data
for k in missing_val_ctype_col:
    trainfile_cleaned.iloc[:, k] = trainfile_cleaned.iloc[:, k].fillna(trainfile_cleaned.iloc[:, k].mode()[0])

for k in missing_val_ntype_col:
    trainfile_cleaned.iloc[:, k] = trainfile_cleaned.iloc[:, k].fillna(trainfile_cleaned.iloc[:, k].median())

# One-Hot Encoding for categorical columns
classification_col = trainfile_cleaned.select_dtypes(include=['object', 'category']).columns

# Apply One-Hot Encoding
trainfile_encoded = pd.get_dummies(trainfile_cleaned, columns=classification_col)

# Display the updated dataframe with encoded columns
# print("After One-Hot Encoding:")
# print(trainfile_encoded.head())

# To check how many columns are created after encoding
# print(f"Total number of columns after encoding: {trainfile_encoded.shape[1]}")

# Assume 'target_column' is the column you want to predict (i.e., your dependent variable)
X = trainfile_encoded.drop('SalePrice', axis=1)  # Features (all columns except the target)
y = trainfile_encoded['SalePrice']  # Target (dependent variable)

# Split the data into training and testing sets (e.g., 80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")

# Random Forest Regressor
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Predicting using test data.
y_pred = model.predict(X_test)
# print(y_pred)

# Calculating mse and rsquare for regression data.
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# Hyper parameter tuning/ cross validation.
# Define the hyperparameter grid to search
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees
    'max_depth': [10, 20, 30],        # Depth of trees
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4]     # Minimum samples required at a leaf node
}

# Initialize the model
model = RandomForestRegressor(random_state=42)

# GridSearchCV to find the best combination of hyperparameters
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

# Fit the grid search model
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print(f"Best Hyperparameters: {best_params}")

# Get the best model from Grid Search or Randomized Search
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Evaluate the performance of the model
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error on Test Set: {mse}")
print(f"R-squared on Test Set: {r2}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Training data shape: (1168, 267)
Testing data shape: (292, 267)
Mean Squared Error: 852273023.2856407
R-squared: 0.8888869912094407
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best Hyperparameters: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Mean Squared Error on Test Set: 853891109.0901171
R-squared on Test Set: 0.888676037234242
